# US National Tourism Data Warehouse from Scratch

In [82]:
import pandas as pd
import numpy as np
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

### Dimension Tables:

    - dim_us_city
    - dim_city_temperature
    - dim_airport
    - dim_state
    - dim_datetime
    - dim_airline

## 1. Dimension Table: `dim_us_city` 

In [74]:
us_city_path  = os.getcwd() + '/datasets/us-cities-demographics.csv'
df_city = pd.read_csv(us_city_path, delimiter=';')
df_city.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


For the purpose of this project we're going to select just a few columns and transform some others. e.g: We are going to group the data by City. There's no need to sum the population of male and females because the source already did that for us. We can corroborate this by choosing a random city and compare with this link: https://suburbanstats.org/

Some columns are going to be discarded like race, n˚ of veterans, avg household size, etc. because don't fit in the purpose of the project.

Columns of `us_city`: 
* id_city (Serial type in database)
* city (Name of the city)
* male population (Group by city)
* female population (Group by city)
* total population (male + female population)

In [75]:
df_city = df_city[['City', 'State', 'Male Population', 'Female Population', 'Total Population']].drop_duplicates()
df_city.head()

,City,State,Male Population,Female Population,Total Population
0,Silver Spring,Maryland,40601.0,41862.0,82463
1,Quincy,Massachusetts,44129.0,49500.0,93629
2,Hoover,Alabama,38040.0,46799.0,84839
3,Rancho Cucamonga,California,88127.0,87105.0,175232
4,Newark,New Jersey,138040.0,143873.0,281913


### Data Cleaning for `us_city` table

Checking for null values in every column

In [77]:
df_city.isnull().any()

City                 False
State                False
Male Population       True
Female Population     True
Total Population     False
dtype: bool

We have columns `Male Population` and `Female Population` with null values. Let's see what are the other values for those rows.

In [78]:
nan_city = df_city[df_city.isnull().T.any().T]
nan_city.head()

,City,State,Male Population,Female Population,Total Population
333,The Villages,Florida,NaN,NaN,72590


We just have one null row. let's change those NaN values using fillna. 

In [80]:
df_city.fillna({'Male Population': 0, 'Female Population': 0}, inplace=True)
df_city.loc[333]

City                 The Villages
State                     Florida
Male Population                 0
Female Population               0
Total Population            72590
Name: 333, dtype: object

In [81]:
df_city.isnull().any()

City                 False
State                False
Male Population      False
Female Population    False
Total Population     False
dtype: bool

## 2. Dimension Table: `dim_city_temperature` 

Let's create a Spark Session and write the data in parquet files filtered by Country. 
For our main dataset (Immigration) we have only data for United States that's why we're going to filter this dataset by country. 

In [87]:
# Create a Spark Session 

spark = SparkSession.builder\
    .appName('weather')\
    .getOrCreate()
df_spark =spark.read.format('csv').option('header', 'true').load(os.getcwd() + '/datasets/GlobalLandTemperaturesByCity.csv')

df_spark.filter("Country = 'United States'").write.partitionBy('City').parquet('weather.parquet')


In [150]:
# Read parquet file 

df = spark.read.parquet('weather.parquet')
df.limit(5).toPandas()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Latitude,Longitude,City
0,1758-03-01,6.422999999999999,3.742,United States,37.78N,93.56W,Springfield
1,1758-04-01,12.14,5.432,United States,37.78N,93.56W,Springfield
2,1758-05-01,16.997999999999998,3.76,United States,37.78N,93.56W,Springfield
3,1758-06-01,22.852,3.5189999999999997,United States,37.78N,93.56W,Springfield
4,1758-07-01,25.195,4.888,United States,37.78N,93.56W,Springfield


Search for null values in AverageTemperature

In [139]:
df.filter('AverageTemperature is null').count()

25765

What's the max date for those values? We need from 2016 so we can delete other null values.

In [140]:
%%time
max_date = df.filter('AverageTemperature is null').agg({'dt':'max'}).collect()[0][0]
min_date = df.filter('AverageTemperature is null').agg({'dt':'min'}).collect()[0][0]

print(f"Max date for null values: {max_date} \nMin date for null values {min_date}")

Max date for null values: 2013-09-01 
Min date for null values 1743-12-01
CPU times: user 9.53 ms, sys: 4.3 ms, total: 13.8 ms
Wall time: 1.15 s


In [141]:
%%time
max_date = df.filter('AverageTemperature is null').select(F.max('dt').alias("MAX")).limit(1).collect()[0].MAX
min_date = df.filter('AverageTemperature is null').select(F.min('dt').alias('MIN')).limit(1).collect()[0].MIN

print(f"Max date for null values: {max_date} \nMin date for null values {min_date}")

Max date for null values: 2013-09-01 
Min date for null values 1743-12-01
CPU times: user 9.51 ms, sys: 4.29 ms, total: 13.8 ms
Wall time: 1.23 s


We need to have in mind that we're working with Big Data. That's why is always good to check other alternatives and figure out how much time and CPU are comsume each method. In the example below, the second one is better, less time and less CPU usage! 

Now, it's time to delete those null values. 

In [151]:
df = df.dropna('any')
df.filter('AverageTemperature is null').count()

0

Change `dt` type column to `DateType`

In [152]:
from datetime import datetime
date_udf = F.udf(lambda x: datetime.strptime(x, "%Y-%m-%d"), T.DateType())
df_date = df.withColumn("dt", date_udf(df.dt))
df_date.limit(5).toPandas()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Latitude,Longitude,City
0,1758-03-01,6.422999999999999,3.742,United States,37.78N,93.56W,Springfield
1,1758-04-01,12.14,5.432,United States,37.78N,93.56W,Springfield
2,1758-05-01,16.997999999999998,3.76,United States,37.78N,93.56W,Springfield
3,1758-06-01,22.852,3.5189999999999997,United States,37.78N,93.56W,Springfield
4,1758-07-01,25.195,4.888,United States,37.78N,93.56W,Springfield
